In [2]:
import numpy as np
from scipy.special import jv, yv
import pandas as pd

# LSRK5 係數 (Carpenter & Kennedy 1994)
A = np.array([0.0,
              -567301805773.0 / 1357537059087.0,
              -2404267990393.0 / 2016746695238.0,
              -3550918686646.0 / 2091501179385.0,
              -1275806237668.0 / 842570457699.0])

B = np.array([1432997174477.0 / 9575080441755.0,
              5161836677717.0 / 13612068292357.0,
              1720146321549.0 / 2090206949498.0,
              3134564353537.0 / 4481467310338.0,
              2277821191437.0 / 14882151754819.0])

C = np.array([0.0, 0.1496590219993, 0.3704009573644, 0.6222557631345, 0.9582821306748])

# 問題參數
n = 8
c = 1.0
alpha = 6.1556542986056666
omega = alpha * c

In [5]:
def exact_solution(r, theta, t, n, alpha, omega):
    """
    解析解 - 矩陣維度 (K, N+1)
    Args:
        r, theta: meshgrid 座標矩陣，形狀 (K, N+1)
        t: 時間
        n, alpha, omega: 問題參數
    Returns:
        u, u_t: 解和時間導數，形狀 (K, N+1)
    """
    hat_u_expanded = hat_u[:, np.newaxis]  # (K, 1) 廣播為 (K, N+1)
    u = hat_u_expanded * np.cos(n * theta - omega * t)
    u_t = omega * hat_u_expanded * np.sin(n * theta - omega * t)
    return u, u_t

def f_r_analytic(r, theta, t, n, alpha, omega):
    """
    徑向項解析形式 - 矩陣維度 (K, N+1)
    """
    hat_u_expanded = hat_u[:, np.newaxis]
    radial_term = (64 / r**2 - alpha**2) * hat_u_expanded
    return radial_term * np.cos(n * theta - omega * t)

def construct_D_matrix(N_plus_1):
    """
    構建傅立葉偽譜微分矩陣
    Args:
        N_plus_1: 角度方向格點數 (N+1)
    Returns:
        D: 一階微分矩陣 (N+1, N+1)
    """
    D = np.zeros((N_plus_1, N_plus_1))
    for i in range(N_plus_1):
        for j in range(N_plus_1):
            if i != j:
                diff = i - j
                D[i, j] = ((-1)**diff) / (2 * np.tan(np.pi * diff / N_plus_1))
    return D

def f_theta_numerical(u, R_inv2, D2):
    """
    角度項數值計算
    Args:
        u: 解矩陣，形狀 (K, N+1)
        R_inv2: 對角矩陣，形狀 (K, K)
        D2: 二階微分矩陣 (N+1, N+1)
    Returns:
        f_theta: 角度項，形狀 (K, N+1)
    """
    return R_inv2 @ u @ D2  # 矩陣乘法，形狀 (K, N+1)

def rhs_system(u, v, r, theta, t, n, alpha, omega, c, r_arr, N_plus_1, D2, R_inv2):
    """
    系統右端項
    Args:
        u, v: 解及其時間導數，形狀 (K, N+1)
        r, theta: 座標矩陣，形狀 (K, N+1)
        其他參數...
    Returns:
        du_dt, dv_dt: 時間導數，形狀 (K, N+1)
    """
    du_dt = v
    dv_dt = f_r_analytic(r, theta, t, n, alpha, omega) + f_theta_numerical(u, R_inv2, D2)
    return du_dt, dv_dt

In [7]:
# 傅立葉方法角度離散化 - 簡化版本
K = 20
CFL = 0.5
T = 2 * np.pi / alpha

# 根據文檔，使用 N+1 個角度點
N_plus_1_list = [64, 96, 128, 160, 192, 224]
errors = []

A_coef = 1.0
B_coef = -A_coef * jv(n, alpha * 1) / yv(n, alpha * 1)

for N_plus_1 in N_plus_1_list:
    D = construct_D_matrix(N_plus_1)
    D2 = D.T @ D.T
    
    # 網格設置 - 使用 K 個內部點
    r = np.array([1 + (k+1) / (K+1) for k in range(K)])  # K 個內部點
    R_inv2 = np.diag(1 / r**2)  # (K, K)
    hat_u = A_coef * jv(n, alpha * r) + B_coef * yv(n, alpha * r)
    theta = np.array([2 * np.pi * n / N_plus_1 for n in range(N_plus_1)])  # θ_n = 2πn/(N+1) (N+1個點)
    dr = 1 / K
    dtheta = 2 * np.pi / N_plus_1
    
    # 矩陣排列：行對應徑向r，列對應角度θ - 產生 (K, N+1) 維度的矩陣
    R, Theta = np.meshgrid(r, theta, indexing='ij')  # R.shape = (K, N+1), Theta.shape = (K, N+1)
    
    # 時間步進
    dt = CFL * min(dtheta, dr)
    num_steps = int(np.ceil(T / dt))
    dt = T / num_steps
    
    # 初始條件
    u, v = exact_solution(R, Theta, 0, n, alpha, omega)
    
    # LSRK5 時間步進
    for step in range(num_steps):
        t_n = step * dt
        u_stage, v_stage = u.copy(), v.copy()
        du, dv = np.zeros_like(u), np.zeros_like(v)
        
        for j in range(5):
            t_stage = t_n + C[j] * dt
            du_dt, dv_dt = rhs_system(u_stage, v_stage, R, Theta, t_stage, n, alpha, omega, c, r, N_plus_1, D2, R_inv2)
            du = A[j] * du + dt * du_dt
            dv = A[j] * dv + dt * dv_dt
            u_stage += B[j] * du
            v_stage += B[j] * dv
        
        u, v = u_stage, v_stage
    
    # 誤差計算
    u_exact, _ = exact_solution(R, Theta, T, n, alpha, omega)
    max_error = np.max(np.abs(u - u_exact))
    errors.append(max_error)

# 收斂率計算
rates = [np.nan]
for i in range(1, len(errors)):
    rate = np.log(errors[i-1] / errors[i]) / np.log(N_plus_1_list[i] / N_plus_1_list[i-1])
    rates.append(rate)

# 結果表格
df = pd.DataFrame({
    "N+1": N_plus_1_list,
    "Error": errors,
    "Rate": rates
})

print("數值結果:")
print(df.to_string(index=False, float_format="%.6e"))

數值結果:
 N+1        Error          Rate
  64 2.651488e-06           NaN
  96 2.748359e-06 -8.849887e-02
 128 2.495033e-06  3.361418e-01
 160 1.055080e-06  3.857092e+00
 192 4.917274e-07  4.187369e+00
 224 2.738131e-07  3.798094e+00
